# WoS dataset filtering
In this notebok, we'll address the problem of filtering out irrelevant papers from our Web of Science-generated dataset. We'll explore a few approaches to this problem, including filtering based on static and dynamic WoS keywords. We'll then evaluate our approaches on a subset of abstracts annotated for whether or not theyshould be kept or removed from the dataset (relevance labeled).

In [2]:
import jsonlines
import pandas as pd

## Reading in the data
We've done some labeling of the dynamic and static keywords, in addition to needing to read in our entire dataset.

In [3]:
### UPDATE with newest version when available
with jsonlines.open('../data/wos_files/core_collection_destol_or_anhydro_ALL_15Dec2023.jsonl') as reader:
    data = []
    for obj in reader:
        data.append(obj)

In [6]:
dynamos = pd.read_csv('../data/wos_files/dynamic_keys_relevance_classified_27Dec2023.csv', index_col=0)
dynamos.head()

,relevant
Aplysia,Y
Molecular & Cell Biology - Pharmacology,N
Immunology,N
Archaeometry,N
Selenium,Y


In [7]:
statics = pd.read_csv('../data/wos_files/static_keys_relevance_classified_27Dec2023.csv', index_col=0)
statics.head()

,relevant
"Engineering, Chemical",N
Science & Technology - Other Topics,N
Immunology,N
Computer Science,N
Construction & Building Technology,N


In [ ]:
## TODO: read in the test set here once it's been finalized